In [ ]:
#@title
# Upload: 
# module14_a.py
# module14_b.py
# electrical_grid_stability_simulated_data.csv

# 14.3. Use Terminal to Transfer to MARCC, Run on MARCC, and Return the Results

In [ ]:
#@title 14.3.1. Login to MARCC and create a project folder (e.g., my_project)
# "ssh youremail@login.marcc.jhu.edu"
# enter "yes"
# enter MARCC verification code from your Google Authenticator application
# enter your MARCC password 
# - example:
#   "ssh mrafiei1@jhu.edu@login.marcc.jhu.edu"
# "mkdir my_project"
# "exit" to go back to your computer

In [ ]:
#@title 14.3.2. Transfer to MARCC
# "scp sorce_files destination_folder"
# - example:
#   "scp *.py *.csv youremail@login.marcc.jhu.edu:~/designated_folder_on_your_marcc_account"
# - example (assming that you a folder on marcc):
#   "scp *.py *.csv mrafiei1@jhu.edu@login.marcc.jhu.edu:~/my_project"

In [ ]:
#@title 14.3.3. Login to MARCC and go to folder my_project
# "ssh youremail@login.marcc.jhu.edu"
# enter "yes"
# enter MARCC verification code from your Google Authenticator application
# enter your MARCC password 
# - example:
#   "ssh mrafiei1@jhu.edu@login.marcc.jhu.edu"
# "cd my_project"

In [ ]:
#@title 14.3.4. Edit module14_b.py using "vim" text editor
# "vim module14_b.py" (and "Enter" if asked)
# type "i" one time to start editing
# Edit file based on your needs
#"Esc" one time and type ":wq!" and "Enter" to save the module14_b.py with the changes
# P.S. you may "Esc" and enter ":q!" and "Enter" to leave module14_b.py without saving any chanegs

In [ ]:
#@title 14.3.5. Run module14_b.py
# "module load Python/3.7.7"
# "python module14_b.py"
# "ls -la" to see the generated files

In [ ]:
#@title 14.3.6. Sumbit jobs
# "bash job_all.sh" to submit all jobs at once to master node (scheduler)
# or you can submit each job seperately using "sbatch your_job"
# use "sqme" to see the sattus of your job and job numbers
# wait until all jobs are scheduled and run

In [ ]:
#@title 14.3.7. Gather the results
# Once all jobs are done, type "zip results.zip result_*" to zip all the results
# type "exit" to exit MARCC and get back to your computer
# type "scp youremail@login.marcc.jhu.edu:~/my_project/*.zip /~" to bring the result.zip file to your computer
# - Example:
#   "scp mrafiei1@jhu.edu@login.marcc.jhu.edu:~/my_project/*.zip /content/"
# type "unzip results.zip" to unzip all the .npy result files

# 14.4. Summarizing the results of runs on MARCC

In [ ]:
#@title 14.4.1. Import necessary libraries + some inputs 
!wget https://raw.githubusercontent.com/mhrafiei/data/main/electrical_grid_stability_simulated_data.csv

import glob
import numpy as np
import pandas as pd
import matplotlib
import itertools # new

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

#############################################################
# Data processing
#############################################################

data = pd.read_csv('electrical_grid_stability_simulated_data.csv')

datain = data.iloc[:,0:12]
dataou = data.iloc[:,-1:]

dataou[dataou == 'stable']   = 1
dataou[dataou == 'unstable'] = 0

datain = datain.values 
dataou = dataou.values

# object to float data
mat = np.empty(dataou.shape,dtype=float)
for i0 in range(dataou.shape[0]): mat[i0,0] = dataou[i0]
dataou = mat

# define some simple keys
data_keys = list(data.keys())
keys = []
for i0 in range(len(data_keys)):
  keys.append('feature {:02d}'.format(i0+1))

# remove the outputs
keys = keys[:13]

# necessary information for combinatory model
num_data      = datain.shape[0]
num_variables = datain.shape[1]

In [ ]:
#@title 14.4.2. Combine the results
num_comb          = 2**12 - 1
model_type        = ['SVC', 'KNC', 'DTC', 'RFC']
RTT               = [0.1,0.2, 0.3, 0.4]
RRS               = 25

result_names = glob.glob('result_*')
result_names.sort()

result   = np.empty((len(RTT), RRS, num_comb, len(model_type)), dtype = dict)
comb_bin = np.zeros((num_comb, 12))

for i0 in range(len(result_names)):
  res_case   = np.load(result_names[i0], allow_pickle = True)
  res_case   = res_case.item()
  comb_case  = res_case['comb_bin']   
  comb_start = res_case['range'][0] 
  comb_end   = res_case['range'][1]
  comb_bin[comb_start:comb_end,:] = comb_case * 1
  result[:,:,comb_start:comb_end,:] = res_case['result']

  print('result # {:03d}'.format(i0) + ' has been retrieved')

In [ ]:
#@title 14.4.3. Result function
def fun_result(result, model_type, comb_bin):
  num_rtt, num_rrs, num_comb, num_model = result.shape
  result_summary                        = {}

  for i3 in range(num_model):
    res_tr = np.empty((num_comb, 1))
    res_te = np.empty((num_comb, 1))

    for i0 in range(num_comb):

      mse_te = np.empty((num_rtt, num_rrs))
      mse_tr = np.empty((num_rtt, num_rrs))

      for i1 in range(num_rtt):
        for i2 in range(num_rrs):
          mse_tr[i1,i2] = result[i1, i2, i0, i3]['acc_tr']
          mse_te[i1,i2] = result[i1, i2, i0, i3]['acc_te']

      res_tr[i0,0] = mse_tr.mean()
      res_te[i0,0] = mse_te.mean()

    mse_tr_sorted  = np.sort(res_tr, axis = 0)[:,0]
    mse_tr_index   = np.argsort(res_tr, axis = 0)[:,0]

    mse_te_sorted  = np.sort(res_te, axis = 0)[:,0]
    mse_te_index   = np.argsort(res_te, axis = 0)[:,0]

    #print(mse_te_index)

    result_summary[model_type[i3]] = {'acc_tr&te'  : [mse_tr_sorted, res_te[mse_tr_index,0]], 
                                      'comb_bin_tr': comb_bin[mse_tr_index,:], 
                                      'acc_te&tr'  : [mse_te_sorted, res_tr[mse_te_index,0]], 
                                      'comb_bin_te': comb_bin[mse_te_index,:]}

  return result_summary


In [ ]:
#@title 14.4.4. Results and summaries
result_summary = fun_result(result, model_type, comb_bin)

#########################################
# Create a report about best combinations
#########################################
df = {}
# add combinations 
for i0 in range(datain.shape[1]):
  comb_best = []
  index     = []
  for i1 in range(len(model_type)):
    comb_best_tr = result_summary[model_type[i1]]['comb_bin_tr'][-1,i0]*1
    comb_best_te = result_summary[model_type[i1]]['comb_bin_te'][-1,i0]*1

    comb_best.append(comb_best_tr)
    comb_best.append(comb_best_te)

  df[keys[i0]] = comb_best

# add mse and data frame index (i.e., row names) 
index       = []
acc_best_tr = [] 
acc_best_te = [] 
for i1 in range(len(model_type)):
  acc_best_tr.append(result_summary[model_type[i1]]['acc_tr&te'][0][-1])
  acc_best_te.append(result_summary[model_type[i1]]['acc_tr&te'][1][-1])

  acc_best_tr.append(result_summary[model_type[i1]]['acc_te&tr'][0][-1])
  acc_best_te.append(result_summary[model_type[i1]]['acc_te&tr'][1][-1])

  index.append(model_type[i1] + ' Training')
  index.append(model_type[i1] + ' Testing')

df['Accuracy Training'] = acc_best_tr
df['Accuracy Testing']  = acc_best_te

df = pd.DataFrame(df, index=index)
df.to_csv('report.csv')
print("Best Combinations and Their Corresponding Accuracies")
print("----------------------------------------------")
print(df)

df_summary = df

#######################################
# Create a report about selection rates
#######################################
col_val    = ['bo-','bs-','bd-','b*-','bo--','bs--','bd--','b*--','bo:','bs:','bd:','b*:', 'bo-.','bs-.','bd-.','b*-.']
for i1 in range(len(model_type)):
  comb_cumsum = np.cumsum(result_summary[model_type[i1]]['comb_bin_tr'][::-1,:]*1, axis = 0)
  comb_denom  = np.repeat(np.expand_dims(np.array(range(1,comb_cumsum.shape[0]+1)), axis = 1), comb_cumsum.shape[1], axis = 1) 
  comb_rate   = comb_cumsum / comb_denom  * 100

  filename = 'Selection_Rates_' +  model_type[i1] + '.csv'
  comb_dict = {}

  # top 1% num
  num_top    = int(comb_rate.shape[0] * 0.01)
  comb_label = [] 
  for i2 in range(num_top,0,-1):
    comb_label.append(str(i2))

  plt.figure(figsize = [14,7])
  for i2 in range(len(keys[:-1])):
    comb_dict[keys[i2]] = comb_rate[:,i2]

    # plot top 10% selection rates
    
    plt.plot(comb_rate[range(num_top,0,-1),i2], 
             col_val[i2], 
             linewidth = 1, 
             markersize = 5,
             markerfacecolor = 'w')

  plt.xlabel('Combination #', fontsize = 20)
  plt.ylabel('Selection Rate (%)', fontsize = 20)
  plt.title(model_type[i1], fontsize = 20)
  plt.xticks(np.arange(num_top), np.arange(num_top,0,-1))
  plt.grid(color=[.75,.75,.75], linestyle='-', linewidth=1, which='both')
  plt.ylim([0,105])
  plt.legend(keys[:-1], fontsize = 14)
  fname = "{}.png".format(model_type[i1])
  plt.savefig(fname, dpi = 150)
  
  #plt.xticks(list(range), ['January', 'February', 'March'],
  df = pd.DataFrame(comb_dict)
  df.to_csv(filename)


In [ ]:
#@title
#!rm *